# 1. Task的优势

ThreadPool相比Thread来说具备了很多优势，但是ThreadPool却又存在一些使用上的不方便。比如：
- ThreadPool不支持线程的取消、完成、失败通知等交互性操作；
- ThreadPool不支持线程执行的先后次序；


以往，如果开发者要实现上述功能，需要完成很多额外的工作，现在，FCL中提供了一个功能更强大的概念：Task。Task在线程池的基础上进行了优化，并提供了更多的API。在FCL4.0中，如果我们要编写多线程程序，Task显然已经优于传统的方式。  
以下是两个简单的任务示例：

In [1]:
using System.Threading;
using System.Threading.Tasks;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
Task t = new Task(() =>
{
    Console.WriteLine("任务开始工作... ...");
    //模拟工作过程
    Thread.Sleep(5000);
});
t.Start();
t.ContinueWith((task) =>
{
    Console.WriteLine($"任务完成，完成时候的状态为： IsCanceled={task.IsCanceled}  IsCompleted={task.IsCompleted}  IsFaulted={task.IsFaulted}");
});

// Task.WaitAll(t);

//模拟另一个任务
for (int i = 7; i > 0; i--)
{
    Console.WriteLine($"==== {i} ====");
    Thread.Sleep(1000);
}

==== 7 ====
任务开始工作... ...
==== 6 ====
==== 5 ====
==== 4 ====
==== 3 ====
任务完成，完成时候的状态为： IsCanceled=False  IsCompleted=True  IsFaulted=False
==== 2 ====
==== 1 ====



(8,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [3]:
public static class TaskClass
{
    public static void TaskMethod(string taskname, int duration)
    {
        Console.WriteLine($"Task [{taskname}] is start... ...");
        Console.WriteLine($"Task [{taskname}] is running on a thread id: {Thread.CurrentThread.ManagedThreadId}, is thread pool thread: {Thread.CurrentThread.IsThreadPoolThread}");
        Thread.Sleep(duration);
        Console.WriteLine($"Task [{taskname}] is stop... ...");
    }

    public static void TaskReport(Task task)
    {
        Console.WriteLine($"任务完成，完成时候的状态为： IsCanceled={task.IsCanceled}  IsCompleted={task.IsCompleted}  IsFaulted={task.IsFaulted}");
    }
}

In [4]:
Task t = new Task(() => TaskClass.TaskMethod("TestTask", 5000));
t.Start();
t.ContinueWith((task) => TaskClass.TaskReport(task));

//模拟另一个任务
for (int i = 7; i > 0; i--)
{
    Console.WriteLine($"==== {i} ====");
    Thread.Sleep(1000);
}

==== 7 ====
Task [TestTask] is start... ...
Task [TestTask] is running on a thread id: 20, is thread pool thread: True
==== 6 ====
==== 5 ====
==== 4 ====
==== 3 ====
Task [TestTask] is stop... ...
任务完成，完成时候的状态为： IsCanceled=False  IsCompleted=True  IsFaulted=False
==== 2 ====
==== 1 ====



(3,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



# 2. Task的用法

## 2.1. 创建任务

### 2.1.1 无返回值的方式

In [5]:
//方式1
Task t1 = new Task(() => TaskClass.TaskMethod("t1", 3000));
t1.Start();
Task.WaitAll(t1);  //阻塞，等待线程结束才能继续

Console.WriteLine("t1 is done.");

Task [t1] is start... ...
Task [t1] is running on a thread id: 5, is thread pool thread: True
Task [t1] is stop... ...
t1 is done.


In [6]:
//方式2
Task.Run(() => TaskClass.TaskMethod("t2", 3000));

Console.WriteLine("t2 is done.");  //直接执行，非阻塞

t2 is done.
Task [t2] is start... ...
Task [t2] is running on a thread id: 20, is thread pool thread: True



(2,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [7]:
//方式3
Task t3 = Task.Factory.StartNew(() => TaskClass.TaskMethod("t3", 3000));
Task.WaitAll(t3);  //阻塞，等待线程结束才能继续

Console.WriteLine("t3 is done.");

Task [t3] is start... ...
Task [t3] is running on a thread id: 5, is thread pool thread: True
Task [t3] is stop... ...
t3 is done.


In [8]:
//方式4
Task.Factory.StartNew(() => TaskClass.TaskMethod("t4", 3000));  //直接异步的方法 

Console.WriteLine("t4 is done.");  //直接执行，非阻塞

t4 is done.
Task [t4] is start... ...
Task [t4] is running on a thread id: 7, is thread pool thread: True



(2,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [9]:
Task t1 = new Task(() => TaskClass.TaskMethod("t1", 3000));
Task t2 = new Task(() => TaskClass.TaskMethod("t2", 3000));
t2.Start();
t1.Start();
Task.WaitAll(t1, t2);

Task.Run(() => TaskClass.TaskMethod("t3", 3000));
Task.Factory.StartNew(() => TaskClass.TaskMethod("t4", 3000));
Task.Factory.StartNew(() => TaskClass.TaskMethod("t5", 3000), TaskCreationOptions.LongRunning);  //标记为长时间运行任务，则任务不会使用线程池，而在单独的线程中运行

Task [t2] is start... ...
Task [t2] is running on a thread id: 5, is thread pool thread: True
Task [t1] is start... ...
Task [t1] is running on a thread id: 20, is thread pool thread: True
Task [t2] is stop... ...
Task [t1] is stop... ...
Task [t4] is start... ...
Task [t4] is running on a thread id: 23, is thread pool thread: True
Task [t3] is start... ...
Task [t3] is running on a thread id: 20, is thread pool thread: True
Task [t5] is start... ...
Task [t5] is running on a thread id: 24, is thread pool thread: False



(7,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.

(8,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.

(9,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



In [10]:
//常规的使用方式

Console.WriteLine("主线程执行业务处理... ...");
//创建任务
Task task = new Task(() =>
{
    Console.WriteLine("使用System.Threading.Tasks.Task执行异步操作... ...");
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine(i);
    }
});
//启动任务，并安排到当前任务队列线程中执行任务(System.Threading.Tasks.TaskScheduler)
task.Start();
Console.WriteLine("主线程执行其他处理... ...");
task.Wait();

主线程执行业务处理... ...
主线程执行其他处理... ...
使用System.Threading.Tasks.Task执行异步操作... ...
0
1
2
3
4
5
6
7
8
9


In [11]:
//async/await的实现方式:

public async static void AsyncFunction()
{
    await Task.Delay(1);
    Console.WriteLine("使用System.Threading.Tasks.Task执行异步操作... ...");
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine(string.Format("AsyncFunction:i={0}", i));
    }
}

Console.WriteLine("主线程执行业务处理... ...");
AsyncFunction();
Console.WriteLine("主线程执行其他处理... ...");
for (int i = 0; i < 10; i++)
{
    Console.WriteLine(string.Format("Main:i={0}", i));
}

主线程执行业务处理... ...
主线程执行其他处理... ...
Main:i=0
Main:i=1
Main:i=2
Main:i=3
Main:i=4
Main:i=5
Main:i=6
Main:i=7
Main:i=8
Main:i=9
使用System.Threading.Tasks.Task执行异步操作... ...
AsyncFunction:i=0
AsyncFunction:i=1
AsyncFunction:i=2
AsyncFunction:i=3
AsyncFunction:i=4
AsyncFunction:i=5
AsyncFunction:i=6
AsyncFunction:i=7
AsyncFunction:i=8
AsyncFunction:i=9
